# Create and save localizations

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('acdcpp/Automatic-Circuit-Discovery/')
sys.path.append('acdcpp/')
# from acdc import TLACDCExperiment
# from acdcpp.ACDCPPExperiment import ACDCPPExperiment

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
import re

# import acdc
# from acdc.TLACDCExperiment import TLACDCExperiment
# from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

import torch
import pickle

device = t.device('cuda') if t.cuda.is_available() else t.device('mps:0')
print(f'Device: {device}')

/Users/aaquibsyed/.pyenv/versions/3.10.0/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/aaquibsyed/.pyenv/versions/3.10.0/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/aaquibsyed/.pyenv/versions/3.10.0/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Device: mps:0


## ACDCPP/EAP

In [5]:
# set up pipeline from acdcpp to edge mask
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

/Users/aaquibsyed/.pyenv/versions/3.10.0/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
torch.cuda.max_memory_allocated(device=device) / 1e9

0.666549248

### IOI Localization

In [5]:
from tasks.ioi.IOITask import IOITask_old, IOITask
# ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device, prep_acdcpp=True, acdcpp_N=25)
ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device, prep_acdcpp=True, acdcpp_N=25, nb_templates=1, prompt_type="ABBA")
ioi_task.set_logit_diffs(model)

ioi_metric = ioi_task.get_acdcpp_metric()
def negative_abs_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ioi_metric(logits))

with t.no_grad():
    clean_logits = model(ioi_task.clean_data.toks)
    corrupt_logits = model(ioi_task.corr_data.toks)
    clean_logit_diff = ioi_task.ave_logit_diff(clean_logits, ioi_task.clean_data).item()
    corrupt_logit_diff = ioi_task.ave_logit_diff(corrupt_logits, ioi_task.corr_data).item()
    print(f'Clean logit diff: {clean_logit_diff:.3f}, Corrupt logit diff: {corrupt_logit_diff:.3f}')

OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
Clean logit diff: 3.040117025375366, Corrupted logit diff: 1.2651995420455933
Clean logit diff: 3.040, Corrupt logit diff: 1.265


In [13]:
from ACDCPPExperiment import ACDCPPExperiment
from cb_utils.mask_utils import get_masks_from_acdcpp_exp
THRESHOLDS = [0.05]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ioi_task.clean_data.toks,
    corr_data=ioi_task.corr_data.toks,
    acdc_metric=negative_abs_ioi_metric,
    acdcpp_metric=ioi_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)
# e=acdcpp_exp.setup_exp(0.0)

# pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

with open(f"localizations/eap/ioi/exp_threshold={THRESHOLDS[0]}", "wb") as f:
    pickle.dump(acdcpp_exp, f)

with open(f"localizations/eap/ioi/gpt2_threshold={THRESHOLDS[0]}", "wb") as f:
    pickle.dump((acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict))

  0%|          | 0/2 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


Edge pruning: 100%|██████████| 1034/1034 [00:00<00:00, 281818.85it/s]


dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 12])


 50%|█████     | 1/2 [00:08<00:08,  8.11s/it]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


100%|██████████| 2/2 [00:15<00:00,  7.86s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 12])


### Induction

In [ ]:
from tasks.induction.InductionTask import InductionTask
ind_task = InductionTask(batch_size=5, tokenizer=model.tokenizer, prep_acdcpp=True, seq_len=15, acdcpp_metric="ave_logit_diff")
ind_task.set_logit_diffs(model)

ind_metric = ind_task.get_acdcpp_metric()
def negative_abs_ind_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ind_metric(logits))

THRESHOLDS = [0.05]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ind_task.clean_data,
    corr_data=ind_task.corr_data,
    acdc_metric=negative_abs_ind_metric,
    acdcpp_metric=ind_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)

acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

# with open(f"localizations/eap/induction/exp_threshold={THRESHOLDS[0]}", "wb") as f:
    # pickle.dump(acdcpp_exp, f)

with open(f"localizations/eap/induction/gpt2_threshold={THRESHOLDS[0]}", "wb") as f:
    pickle.dump((acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict))

### Sports

In [3]:
threshold = 0.001
import pickle
with open('localizations/eap/eap_sports/1000_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

eap_edges = graph.top_edges(n=1000, threshold=threshold)
# eap_edges = set()
# for i in range(eap_scores.shape[0]):
#     for j in range(eap_scores.shape[1]):
#         if eap_scores[i, j] > threshold:
            # eap_edges.add((get_node_name(graph.node_names[i], show_full_index=False), get_node_name(graph.node_names[j, show_full_index=False))))
print(eap_edges)

[('head.0.2', 'mlp.0', 0.009443754330277443), ('head.0.14', 'mlp.0', 0.006188404746353626), ('mlp.6', 'mlp.8', 0.00573932658880949), ('mlp.0', 'mlp.2', 0.005653967149555683), ('head.16.20', 'mlp.16', -0.005345507059246302), ('head.16.20', 'head.17.30.v', 0.005315450485795736), ('mlp.0', 'head.1.16.k', -0.005109565332531929), ('head.14.14', 'mlp.15', -0.004921694286167622), ('mlp.15', 'head.16.20.k', -0.004847021773457527), ('mlp.6', 'mlp.15', -0.004780464340001345), ('mlp.10', 'head.16.20.k', 0.004766407422721386), ('mlp.6', 'head.16.20.k', 0.004757486749440432), ('mlp.0', 'head.1.15.k', -0.004494336899369955), ('mlp.0', 'mlp.5', -0.004315529949963093), ('mlp.0', 'mlp.4', -0.004094669129699469), ('head.16.20', 'mlp.18', 0.004019735846668482), ('head.0.30', 'mlp.0', 0.0039080469869077206), ('mlp.0', 'mlp.6', -0.0038432518485933542), ('mlp.8', 'mlp.9', 0.00376768596470356), ('head.16.20', 'head.21.9.v', 0.003543522208929062), ('mlp.9', 'mlp.11', 0.003415714716538787), ('mlp.6', 'head.21.

In [4]:
len(graph.eap_scores.flatten())

3277824

In [5]:
"""
Convert format:
want: {((3, 'm3'), (3, 'a3.0')),
 ((4, 'm4'), (0, 'm0')),
 ((4, 'm4'), (3, 'a3.0')),
 ((5, 'a5.9'), (0, 'm0')),}

have:
[('mlp.0', 'mlp.2', 0.005653967149555683),
('head.0.14', 'mlp.0', 0.006188404746353626),]
...
"""
from cb_utils.mask_utils import get_formatted_edges_from_eap, get_masks_from_eap_exp
# formatted_eap_edges = get_formatted_edges_from_eap(eap_edges)
# formatted_eap_edges
with open('localizations/eap/eap_sports/1000_graph.pkl', 'rb') as f:
    graph = pickle.load(f)
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_eap_exp(graph, threshold=0.001, num_layers=32, num_heads=32)

In [6]:
(acdcpp_mask_dict['m31'] == 0).sum()

tensor(5)

In [47]:
from tasks import InductionTask
ind_task = InductionTask(batch_size=16, tokenizer=model.tokenizer, prep_acdcpp=True, seq_len=10, acdcpp_metric="ave_logit_diff")
ind_task.set_logit_diffs(model)
print(ind_task.clean_logit_diff, ind_task.corrupted_logit_diff)

12.292320251464844 1.4027974605560303


In [48]:
ind_metric = ind_task.get_acdcpp_metric()
def negative_abs_ind_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ind_metric(logits))

with t.no_grad():
    clean_logits = model(ind_task.clean_data.cuda())
    corrupt_logits = model(ind_task.corr_data.cuda())
    clean_logit_diff = ind_task.ave_logit_diff(clean_logits, ind_task.clean_data).item()
    corrupt_logit_diff = ind_task.ave_logit_diff(corrupt_logits, ind_task.corr_data).item()
    
print(ind_metric(clean_logits))
print(ind_metric(corrupt_logits))

In [51]:
from ACDCPPExperiment import ACDCPPExperiment
from cb_utils.mask_utils import get_masks_from_acdcpp_exp
THRESHOLDS = [0.05]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ind_task.clean_data,
    corr_data=ind_task.corr_data,
    acdc_metric=negative_abs_ind_metric,
    acdcpp_metric=ind_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)
# e=acdcpp_exp.setup_exp(0.0)

# pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


100%|██████████| 1/1 [00:08<00:00,  8.35s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 5, 0, 1, 2, 3, 4, 6, 12])


## Causal Tracing

In [ ]:
# set up pipeline from acdcpp to edge mask
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

## Induction

In [ ]:
from localizations.causal_tracing.causal_tracing import get_causal_tracing_components
from tasks.induction import InductionTask

ind_task = torch.stack(
    InductionTask(batch_size=16, tokenizer=model.tokenizer).train_data,
    dim=0
)

ind_prompts = model.tokenizer.batch_decode(ind_task[:, :-1])
ind_answers = model.tokenizer.batch_decode(ind_task[:, -1])

data = {
    'text': ind_prompts,
    'IO': ind_answers
}

components = get_causal_tracing_components(
    model,
    data,
    n=100
)

## IOI

In [8]:
# from localizations.causal_tracing.causal_tracing import get_causal_tracing_components
# from tasks.ioi.IOITask import IOITask_old, IOITask

# ioi_data = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device).ioi_data

# components = get_causal_tracing_components(
#     model,
#     ioi_data,
#     n=100  
# )

# with open(f"localizations/causal_tracing/ioi/gpt2_threshold=all", "wb") as f:
#     pickle.dump(components)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Loading models

In [58]:
import pickle
from cb_utils.transformer import DemoTransformer
from cb_utils.models import load_demo_gpt2, tokenizer
means_ioi = True
if means_ioi:
    with open("data/gpt2_ioi_abc_means.pkl", "rb") as f:
        means = pickle.load(f)[0]
else:
    with open("data/gpt2_means.pkl", "rb") as f:
        means = pickle.load(f)[0]

edge_masks = True
weight_masks_attn = True
weight_masks_mlp = True
train_base_weights = True
localize_acdcpp = True

# if edge_masks is True, then have mask_dict_superset be acdcpp_mask_dict
mask_dict_superset = None if not edge_masks else acdcpp_mask_dict
# model = load_demo_gpt2(means=means, mask_dict_superset=acdcpp_mask_dict)
if localize_acdcpp:
    weight_mask_attn_dict = acdcpp_weight_mask_attn_dict if weight_masks_attn else None
    weight_mask_mlp_dict = acdcpp_weight_mask_mlp_dict if weight_masks_mlp else None
    base_weight_attn_dict = acdcpp_weight_mask_attn_dict if train_base_weights else None
    base_weight_mlp_dict = acdcpp_weight_mask_mlp_dict if train_base_weights else None

else:
    weight_mask_attn_dict = None
    weight_mask_mlp_dict = None
    base_weight_attn_dict = None
    base_weight_mlp_dict = None

# model = load_demo_gpt2(means=False, edge_masks=edge_masks, mask_dict_superset=mask_dict_superset, weight_masks_attn=weight_masks_attn, weight_masks_mlp=weight_masks_mlp, weight_mask_attn_dict=weight_mask_attn_dict, weight_mask_mlp_dict=weight_mask_mlp_dict, train_base_weights=train_base_weights, base_weight_attn_dict=base_weight_attn_dict, base_weight_mlp_dict=base_weight_mlp_dict)

Using device: cuda:0


In [7]:
from cb_utils.models import load_demo_pythia
model = load_demo_pythia(means=False, )

Using device: cuda:0
Loaded pretrained model pythia-2.8b into HookedTransformer


In [8]:
reference_pythia =  HookedTransformer.from_pretrained(
        'pythia-2.8b',
        fold_ln=False,
        center_writing_weights=False,
        center_unembed=False,
        default_padding_side="left",
        device='cuda'
    )
tokenizer = reference_pythia.tokenizer

Loaded pretrained model pythia-2.8b into HookedTransformer


In [9]:
pythia_tokenizer = reference_pythia.tokenizer

In [14]:
# compare model outputs
test_input = t.tensor(pythia_tokenizer.encode("The quick brown fox jumps over the lazy")).unsqueeze(0).cuda()
assert (model(test_input)[0][0, -1] - reference_pythia(test_input)[0, -1]).abs().max() < 1e-5, (model(test_input)[0][0, -1] - reference_pythia(test_input)[0, -1]).abs().max()

AssertionError: tensor(1.8776, device='cuda:0', grad_fn=<MaxBackward1>)

In [15]:
model(test_input)[0][0, -1]

tensor([ 8.5317, -2.0861,  9.2626,  ..., -2.3424, -2.3985, -2.5804],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [17]:
reference_pythia(test_input)[0, -1]

tensor([ 6.6789, -3.9404,  7.4135,  ..., -4.1955, -4.2537, -4.4353],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [11]:
pythia_tokenizer.decode(torch.argmax(model(test_input)[0][0, -1]))

' dog'

In [12]:
pythia_tokenizer.decode(torch.argmax(reference_pythia(test_input)[0, -1]))

' dog'

In [31]:
from tasks.induction.InductionTask import InductionTask, InductionTask_Uniform

ind_uniform_task = InductionTask_Uniform(batch_size=16, tokenizer=tokenizer, prep_acdcpp=False, seq_len=15, uniform_over="rep_tokens")
ind_uniform_task.get_test_loss(model)

tensor(16., device='cuda:0')


tensor(14.1681, device='cuda:0')

In [25]:
ind_task.get_test_loss(model)

tensor(0.6161, device='cuda:0')

In [64]:
torch.cuda.memory_allocated(device=device) / 1e9

16.789679616

## Test that gradients flow correctly

In [73]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape, param.requires_grad)
    # print(name, param.shape, param.requires_grad)

output_mask torch.Size([157]) True
blocks.0.edge_mask_attentions torch.Size([1, 12]) True
blocks.0.edge_mask_mlp torch.Size([13]) True
blocks.0.attn.W_Q torch.Size([12, 768, 64]) True
blocks.0.attn.b_Q torch.Size([12, 64]) True
blocks.0.attn.W_K torch.Size([12, 768, 64]) True
blocks.0.attn.b_K torch.Size([12, 64]) True
blocks.0.attn.W_V torch.Size([12, 768, 64]) True
blocks.0.attn.b_V torch.Size([12, 64]) True
blocks.0.attn.W_O torch.Size([12, 64, 768]) True
blocks.0.attn.b_O torch.Size([768]) True
blocks.0.attn.weight_mask_W_Q torch.Size([12, 768, 64]) True
blocks.0.attn.weight_mask_W_K torch.Size([12, 768, 64]) True
blocks.0.attn.weight_mask_W_V torch.Size([12, 768, 64]) True
blocks.0.attn.weight_mask_W_O torch.Size([12, 64, 768]) True
blocks.0.mlp.W_in torch.Size([768, 3072]) True
blocks.0.mlp.b_in torch.Size([3072]) True
blocks.0.mlp.W_out torch.Size([3072, 768]) True
blocks.0.mlp.b_out torch.Size([768]) True
blocks.0.mlp.weight_mask_W_in torch.Size([768, 3072]) True
blocks.0.mlp.w

In [74]:
model.train()
batch_size = 8
ioi = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False)
loss = ioi.get_train_loss(model)
loss.backward()

In [75]:
losses = []
for i in range(10):
    losses.append(ioi.get_train_loss(model))
    print(torch.cuda.memory_allocated(device=device) / 1e9)

7.701484032
9.50002944
11.296608768
13.095366144
14.886672896
16.662592
18.435150336
20.20689152
21.983021568
23.762801152


In [68]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if param.requires_grad: # and "edge" in name:
        # check if param.grad is all zeros
        if param.grad is not None and param.grad.sum() != 0:
            print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        else:
            print(f"{name} grad is all zeros")

output_mask grad is all zeros
blocks.0.edge_mask_attentions grad is all zeros
blocks.0.edge_mask_mlp grad is all zeros
blocks.0.attn.W_Q grad is not all zeros, param.grad.norm()=tensor(0.1548, device='cuda:0')
blocks.0.attn.b_Q grad is not all zeros, param.grad.norm()=tensor(0.0549, device='cuda:0')
blocks.0.attn.W_K grad is not all zeros, param.grad.norm()=tensor(0.2216, device='cuda:0')
blocks.0.attn.b_K grad is not all zeros, param.grad.norm()=tensor(1.4179e-08, device='cuda:0')
blocks.0.attn.W_V grad is not all zeros, param.grad.norm()=tensor(1.2717, device='cuda:0')
blocks.0.attn.b_V grad is not all zeros, param.grad.norm()=tensor(0.7394, device='cuda:0')
blocks.0.attn.W_O grad is not all zeros, param.grad.norm()=tensor(2.7787, device='cuda:0')
blocks.0.attn.b_O grad is not all zeros, param.grad.norm()=tensor(1.2196, device='cuda:0')
blocks.0.mlp.W_in grad is not all zeros, param.grad.norm()=tensor(1.8217, device='cuda:0')
blocks.0.mlp.b_in grad is not all zeros, param.grad.norm()

### Check if correct MLPs flow

In [29]:
for node in acdcpp_nodes:
    if "m" in node[1]:
        print(node)

(2, 'm2')
(10, 'm10')
(5, 'm5')
(3, 'm3')
(7, 'm7')
(1, 'm1')
(9, 'm9')
(8, 'm8')
(4, 'm4')
(0, 'm0')
(-1, 'embed')
(6, 'm6')


In [31]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if "mlp" in name:
        # check if param.grad is all zeros
        if param.grad is not None and param.grad.sum() != 0:
            print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        else:
            print(f"{name} grad is all zeros")

blocks.0.edge_mask_mlp grad is all zeros
blocks.0.edge_mask_mlp_baseline grad is all zeros
blocks.0.edge_mask_mlp_frozen grad is all zeros
blocks.0.mlp.W_in grad is not all zeros, param.grad.norm()=tensor(1.5009, device='cuda:0')
blocks.0.mlp.b_in grad is not all zeros, param.grad.norm()=tensor(0.3840, device='cuda:0')
blocks.0.mlp.W_out grad is not all zeros, param.grad.norm()=tensor(1.9580, device='cuda:0')
blocks.0.mlp.b_out grad is not all zeros, param.grad.norm()=tensor(0.2079, device='cuda:0')
blocks.1.edge_mask_mlp grad is all zeros
blocks.1.edge_mask_mlp_baseline grad is all zeros
blocks.1.edge_mask_mlp_frozen grad is all zeros
blocks.1.mlp.W_in grad is not all zeros, param.grad.norm()=tensor(2.1881, device='cuda:0')
blocks.1.mlp.b_in grad is not all zeros, param.grad.norm()=tensor(0.4780, device='cuda:0')
blocks.1.mlp.W_out grad is not all zeros, param.grad.norm()=tensor(2.0281, device='cuda:0')
blocks.1.mlp.b_out grad is not all zeros, param.grad.norm()=tensor(0.2563, device=

### Check if individual attention heads have gradients

In [76]:
for node in acdcpp_nodes:
    if "a9" in node[1]:
        print(node)

(9, 'a9.7')
(9, 'a9.9')
(9, 'a9.6')
(9, 'a9.8')


In [81]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if param.requires_grad and "9.attn.weight" in name:
    # if param.requires_grad and "9.attn.b" in name:
        print(param.shape)
        # if param.grad is not None and param.grad.sum() != 0:
        #     print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        # else:
        #     print(f"{name} grad is all zeros")
        print(f"{param.grad[3].norm()=}")
        print(f"{param.grad[4].norm()=}")
        print(f"{param.grad[7].norm()=}")
        print(f"{param.grad[8].norm()=}")
        print(f"{param.grad[11].norm()=}")
        print()


torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.1051, device='cuda:0')
param.grad[8].norm()=tensor(0.0287, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.1273, device='cuda:0')
param.grad[8].norm()=tensor(0.0335, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.0570, device='cuda:0')
param.grad[8].norm()=tensor(0.0485, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 64, 768])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.030

## Start Mask Learning

In [ ]:
from tasks import IOITask, SportsTask, OWTTask
batch_size = 64
ioi = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False, prompt_type="ABBA", nb_templates=1, template_start_idx=0)
sports = SportsTask(batch_size=batch_size, tokenizer=tokenizer, device=device)
owt = OWTTask(batch_size=batch_size, tokenizer=tokenizer, device=device)

ioi_ood = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False, prompt_type="ABBA", nb_templates=1, template_start_idx=1) # different template

train_tasks = {"ioi": ioi, "owt": owt}
task_weights = {"ioi": -.2, "owt": 1} # I think means preserve OWT, corrupt IOI
eval_tasks = {"ioi": ioi, "sports": sports, "owt": owt}

/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
mask_params = []
param_names = []
for name, p in model.named_parameters():
    if p.requires_grad:
        param_names.append(name)
        mask_params.append(p)

In [ ]:
from cb_utils.learn_mask import train_masks

epochs_left = 500
steps_per_epoch = 10
lr = .05 # free
weight_decay = 0
evaluate_every = 1
discretize_every = 50 # 5 # free
threshold = 0.5
use_wandb = False
edge_mask_reg_strength = None
weight_mask_reg_strength = 10

wandb_config = {"edge_masks": edge_masks, "weight_masks_attn": weight_masks_attn, "weight_masks_mlp": weight_masks_mlp, "epochs": epochs_left, "steps_per_epoch": steps_per_epoch, "lr": lr, "weight_decay": weight_decay, "evaluate_every": evaluate_every, "discretize_every": discretize_every, "threshold": threshold, "edge_mask_reg_strength": edge_mask_reg_strength, "weight_mask_reg_strength": weight_mask_reg_strength}

optimizer = torch.optim.AdamW(mask_params, lr=lr, weight_decay=weight_decay)
train_masks(model, tasks=train_tasks, optimizer=optimizer, num_epochs=epochs_left, steps_per_epoch=steps_per_epoch,
            # param_names=param_names, mask_params=mask_params, 
            task_weights=task_weights, eval_tasks=eval_tasks, evaluate_every=evaluate_every, discretize_every=discretize_every, threshold=threshold, edge_mask_reg_strength=edge_mask_reg_strength, weight_mask_reg_strength=None, verbose=False, use_wandb=use_wandb, wandb_config=wandb_config)

wandb: Currently logged in as: philliphguo. Use `wandb login --relogin` to force relogin


  6%|▋         | 32/501 [10:48<2:38:21, 20.26s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_93609/3197613862.py", line 15, in <module>
    train_masks(model, tasks=train_tasks, optimizer=optimizer, num_epochs=epochs_left, steps_per_epoch=steps_per_epoch,
  File "/data/phillip_guo/mechanistic-unlearning/cb_utils/learn_mask.py", line 178, in train_masks
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/torch/_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/torch/autograd/__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call l

In [ ]:
import pickle
with open(f"masks/trained_mask_params_{epochs_left=}_{edge_mask_reg_strength=}.pkl", "wb") as f:
    pickle.dump(mask_params, f)

In [ ]:
for name, p in zip(param_names, mask_params):
    if p.requires_grad:
        # print(name, p)
        # count how many zeros in p
        print(torch.sum(p == 0))

tensor(8, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(6, device='cuda:0')
tensor(0, device='cuda:0')
tensor(12, device='cuda:0')
tensor(0, device='cuda:0')
